<a href="https://colab.research.google.com/github/byerHu/tf_practice/blob/master/CNN_mnist%E5%9B%BE%E7%89%87%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 加载数据
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

# 每个批次的大小
batch_size = 100

# 计算一个有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

# 初始化偏置值
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# 池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
# 定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

# 改变x的格式转为4D的向量[batch,in_height,in_width,in_channels]
x_image = tf.reshape(x,[-1,28,28,1]) 

# 初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32]) # 采用5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_variable([32]) # 每一个卷积核对应一个偏置值

# 把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 进行max池化

# 初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([5,5,32,64]) # 5*5的采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([64]) # 每个卷积核对应一个偏置值

# 把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu函数
h_conv2  = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 28*28的图片第一次卷积之后还是28*28,第一次池化之后变为了14*14
# 第二次卷积后为14*14，第二次池化后变为了7*7
# 经过上面的操作后得到了64张7*7平面

# 初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*64,1024]) # 上一层7*7*64,全连接层有1024个神经元
b_fc1 = bias_variable([1024])

# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
# 求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

# keep_prob用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2  = bias_variable([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

# 最小化损失
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# 将结果放入一个布尔列表当中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1)) 

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter "+str(epoch)+", Testing Accuracy: "+str(acc))
        

Iter 0, Testing Accuracy: 0.9534
Iter 1, Testing Accuracy: 0.9714
Iter 2, Testing Accuracy: 0.9774
Iter 3, Testing Accuracy: 0.9815
Iter 4, Testing Accuracy: 0.9838
Iter 5, Testing Accuracy: 0.9856
Iter 6, Testing Accuracy: 0.9852
Iter 7, Testing Accuracy: 0.9865
Iter 8, Testing Accuracy: 0.9872
Iter 9, Testing Accuracy: 0.9907
Iter 10, Testing Accuracy: 0.9895
Iter 11, Testing Accuracy: 0.9885
Iter 12, Testing Accuracy: 0.9879
Iter 13, Testing Accuracy: 0.9888
Iter 14, Testing Accuracy: 0.9893
Iter 15, Testing Accuracy: 0.9905
Iter 16, Testing Accuracy: 0.99
Iter 17, Testing Accuracy: 0.9913
Iter 18, Testing Accuracy: 0.9905
Iter 19, Testing Accuracy: 0.9911
Iter 20, Testing Accuracy: 0.9918
